In [2]:
%%capture 
import csv
from torch.utils.data import Dataset
import torch
from sklearn.model_selection import train_test_split
import numpy as np
from bs4 import BeautifulSoup
import string
import spacy
import jsonlines
import json
import re
import torch.nn as nn
from torch.nn.utils.rnn import pad_packed_sequence,pack_padded_sequence,pad_sequence
import torch.nn.functional as F
import torch.optim as optim
import fasttext
from torch.utils.data import SubsetRandomSampler,DataLoader,Subset
from torchtext.vocab import GloVe
from tqdm import tqdm

EMBED_DIM = 300
HIDDEN_DIM = 128

USE_TRAINABLE_EMBEDDINGS = True

PATIENCE_PARAMETER = 4
VALIDATION_LOSS_COMPUTE_STEP = 1


FILTER_SIZES = [3,4,5,6,7]
NUM_FILTERS = 50

device_cpu = torch.device('cpu')
device_fast = torch.device('cpu')

if torch.has_mps:
    device_fast = torch.device('mps')
elif torch.has_cuda:
    device_fast = torch.device('cuda')

#torch.manual_seed(0)
#np.random.seed(0)
nlp = spacy.load('en_core_web_sm')
glove = GloVe()

from torchtext.vocab import vocab
glove_vocab = vocab(glove.stoi)
glove_vocab.insert_token("<unk>",2196015)
glove_vocab.set_default_index(2196015)
pretrained_embeddings = glove.vectors
pretrained_embeddings = torch.cat((pretrained_embeddings,torch.zeros(1,300)))

glove.stoi = glove_vocab.get_stoi()
glove.itos = glove_vocab.get_itos()

glove.vectors = pretrained_embeddings

In [3]:

#punctuation_words = open('punct.txt','w')
#punct_in_file = set()
#final_words = open('words.txt','w')
#punctuations = set(list(string.punctuation))
#vocab_words = set()

def preprocess_text(text):
        
    text = re.sub(r'<br /><br />',".",text)
    text = BeautifulSoup(text,'lxml').get_text().strip()
    text = text.lower()

    #text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r"http\S+", "", text)
    text = ' '.join(re.findall(r"[\w']+|[.,!;/\"]", text))
    
    new_text = []
    for word in text.split():
        if word == '':
            continue
        new_text.append(word)
        #if '.' in word and len(word)>1:
            #if 'http' in word:
            #    words = word.split('.')
            #    words[1] = ''.join(words[1:])
            #    words = [words[0],words[1]]
            #else:
            #    words = word.split('.')
           
        #    words = word.split('.')
        #    for w in words:
        #        new_text.append(w)
        #else:
    
    text = ' '.join(new_text)
    words = nlp(text)
    text =  " ".join([token.text for token in words if not token.is_punct or token.text=='/' or token.text=="\"" or token.text=="."]).strip()
    #review =  " ".join([re.sub(r'^https?:\/\/.*[\r\n]*', '', token.text, flags=re.MULTILINE) for token in words])
    #review  = " ".join(new_text)
    new_words = []
    for word in text.split(" "):
        
        #vocab_words.add(word)
        if word == 'n\'t':
            if len(new_words) > 1:
                new_words[-1] = new_words[-1] + word
            else:
                new_words.append(word)
        else:
            new_words.append(word)
    text = " ".join(new_words)
    return text


In [3]:
preprocessed_dataset = []
train_dataset_labels = []
with open("./Train dataset.csv") as csvfile:
    csvFile = csv.reader(csvfile)
    next(csvFile)
    json_writer = jsonlines.open('processed_dataset.jsonl','w')

    for line in csvFile:
        processed_text = preprocess_text(line[0])
        label = 1.0 if line[1] == 'positive' else 0.0
        train_dataset_labels.append(label)
        json_writer.write({"text":processed_text,"label":label})
        preprocessed_dataset.append({"text":processed_text,"label":label})
    
    json_writer.close()

train_dataset_labels = np.array(train_dataset_labels)
#final_words.write(str(vocab_words))
#final_words.close()
#file.close()
#punctuation_words.close()


/Users/depressedcoder/environments/gymenv/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [4]:
preprocessed_dataset = []
train_dataset_labels = []
with open('processed_dataset.jsonl') as f:
    for line in f:
        sample = json.loads(line)
        train_dataset_labels.append(sample['label'])
        preprocessed_dataset.append(sample)
train_dataset_labels = np.array(train_dataset_labels)


In [5]:
def getWordEmbeddingforText(text,glove=glove):
    
    length = 0
    words = []
    for word in text.split(' '):
        length+=1
        word_embedding = glove[word]
        words.append(word_embedding)
    # MODIFY CODE HERE FOR CNN ARBIRATORY FILTER SIZE
    return torch.stack(words),length

In [6]:
processed_dataset = []

for review in preprocessed_dataset:
    embedding,length = getWordEmbeddingforText(review['text'])
    processed_dataset.append({'text': embedding,'length': length,'label' : review['label']})
 

In [7]:
class ReviewDataSet(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

dataset = ReviewDataSet(processed_dataset)

In [8]:
# Train and Validation split and an equal distriubition of classes
train_idx,valid_idx = train_test_split(np.arange(train_dataset_labels.shape[0]), 
    test_size=0.2,
    shuffle= True,
    stratify= train_dataset_labels,
    random_state=0
)

def collate_function(batch_data):
    inputs = [b['text'] for b in batch_data]
    lengths = [b['length'] for b in batch_data]
    labels = torch.tensor([b['label'] for b in batch_data])

    labels = labels.unsqueeze(1)
    inputs = pad_sequence(inputs,batch_first=True)
    return  {'input' : inputs , 'lengths': lengths , 'labels' : labels }

    

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
train_dataloader = DataLoader(dataset,64,sampler=train_sampler,collate_fn=collate_function)
valid_dataloader = DataLoader(dataset,64,sampler=valid_sampler,collate_fn=collate_function)

In [9]:
class DAN(nn.Module):
    
    def __init__(self,embed_dim=EMBED_DIM,hidden_dim = HIDDEN_DIM, droput_prob = 0.3, train_device = device_cpu):
        super().__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(self.embed_dim,self.hidden_dim)
        #self.fc = nn.Linear(self.hidden_dim,self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim,1)
        self.sigmoid  = nn.Sigmoid()
        self.word_dropout_prob = droput_prob

        self.train_device = train_device

        #if(word_vector_types_count > 1) :
        #   self.importance_weights = nn.Linear(word_vector_types_count*EMBED_DIM,word_vector_types_count)
        

    def forward(self,inp,inp_len):
        
    
        inp_mask = torch.ones((inp.shape[0],inp.shape[1]))
        for i  in range(inp.shape[0]):
            inp_mask[i,inp_len[i]:] = 0.0

        inp_mask = inp_mask.to(self.train_device)

        inp_lengths = torch.sum(inp_mask,-1,keepdim=True).int()
        inp_lengths = inp_lengths.to(self.train_device)
        #if word_vector_types_count > 1:
        #    input = inp.view((inp.shape[0],inp.shape[1],-1))
        #    out = self.importance_weights(input)
        #    weights = F.softmax(out,dim=2).unsqueeze(3)
        #    weights_multiplied_vector = weights * inp
        #    inp = torch.sum(weights_multiplied_vector,dim=2)

        total = torch.sum(inp*(inp_mask.unsqueeze(2)),axis=1)
        vector_average = total / inp_lengths
        ans = F.relu(self.fc1(vector_average))
        #ans = F.relu(self.fc(ans))
        ans = self.sigmoid(self.fc2(ans))
        return ans
dan = DAN(train_device=device_cpu)


In [10]:
class RNNModel(nn.Module):

    def __init__(self,
            embed_dim=EMBED_DIM,hidden_dim =HIDDEN_DIM,bidirectional=False,
            rnn_type = 'gru',num_layers=1,rnn_dropout = 0.4,fc_dropout = 0.3):
        
        super().__init__()
        
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = rnn_dropout
        self.fc_dropout = nn.Dropout(fc_dropout)

        self.rnn = None
        
        if rnn_type == 'gru':
            self.rnn  = nn.GRU(input_size = self.embed_dim,
                hidden_size = self.hidden_dim,
                num_layers = self.num_layers,
                batch_first = True,
                dropout = self.dropout,
                bidirectional = self.bidirectional
            )
        elif rnn_type == 'rnn':
            self.rnn = nn.RNN(input_size = self.embed_dim,
                hidden_size = self.hidden_dim,
                num_layers = self.num_layers,
                batch_first = True,
                dropout = self.dropout,
                bidirectional = self.bidirectional)
        

        self.bidirectional_factor = 2 if self.bidirectional else 1
        self.dnn_input_size= self.bidirectional_factor * self.num_layers * self.hidden_dim
        self.fc_list = []

        log_base_2 = np.log2(self.dnn_input_size)
        nearest_power_2 = int(log_base_2)

        if(float(nearest_power_2) != log_base_2):
            self.fc_list.append(nn.Linear(self.dnn_input_size,2**nearest_power_2))

        while nearest_power_2 > 7 :
            self.fc_list.append(nn.Linear(2**(nearest_power_2),2**(nearest_power_2-1)))
            nearest_power_2-=1
        
        self.fc_list.append(nn.Linear(128,1))
        self.fc = nn.ModuleList(self.fc_list)

        #self.fc1 = nn.Linear(self.dnn_input_size,128)
        #self.fc2 = nn.Linear(128,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x,x_len):

        packed_input = pack_padded_sequence(x,x_len,batch_first=True,enforce_sorted=False)
        packed_output,hidden = self.rnn(packed_input)
        output,output_lengths = pad_packed_sequence(packed_output)
        #hidden = hidden.squeeze()
        hidden = torch.permute(hidden,(1,0,2))
        hidden = hidden.contiguous().view((hidden.shape[0],-1))
        out = hidden
        for i,l in enumerate(self.fc):
            if i!=(len(self.fc_list)-1):
                out = self.fc_dropout(l(out))
            
        #out = self.fc_dropout(self.fc1(hidden))
        ans = self.sigmoid(self.fc_list[len(self.fc_list)-1](out))
        return ans



In [ ]:
sent = RNNModel(bidirectional=True,num_layers=1)

In [21]:
# ADD Dropout Term

class CNNModel(nn.Module):

    def __init__(self, embed_dim=EMBED_DIM,hidden_dim = HIDDEN_DIM,filter_sizes = FILTER_SIZES, n_filters = NUM_FILTERS,dropout = 0.2):
        super().__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.filter_sizes = filter_sizes
        self.num_filters = n_filters
        self.dropout = nn.Dropout(p=dropout)
        
        self.modulelist = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
            out_channels=self.num_filters,
            kernel_size= self.filter_sizes[i]
            )
    
        for i in range(len(self.filter_sizes))])
        
        self.fc1 = nn.Linear(self.num_filters*len(self.filter_sizes),hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x : torch.Tensor,xlen = None):
        
        permuted_x = x.permute(0,2,1)
        x_conv_list = [F.relu(conv(permuted_x)) for conv in self.modulelist]
        x_max_pool_list = [F.max_pool1d(x_conv,kernel_size=x_conv.shape[2]) for x_conv in x_conv_list]
        
        feature = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_max_pool_list],dim=1)
        
        out = self.dropout(F.relu(self.fc1(feature)))
        out = self.fc2(out)
        return self.sigmoid(out)



cnnmodel = CNNModel(10,2,FILTER_SIZES,NUM_FILTERS)    

In [25]:
def length_after_1d_convolution(x,filter_size,max_pool_size):

    def fn(sent_len,a=filter_size,pool_size = max_pool_size):
        conv_out = sent_len - a +1
        return int(conv_out/pool_size)

    ans = list(map(fn,x))
    return ans

class EnsembleModel(nn.Module):

    def __init__(self,embed_dim = EMBED_DIM,hidden_dim = HIDDEN_DIM,filter_size = 3,num_filters=256,max_pool_size = 3,dropout = 0.3,
        rnn_bidirectional = True,
        rnn_num_layers = 1,
        rnn_dropout = 0.0
    ):
        super().__init__()
        
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.filter_size = filter_size
        self.max_pool_size = max_pool_size
        self.dropout = dropout

        self.conv = nn.Conv1d(self.embed_dim,num_filters,self.filter_size)
        self.pool = nn.MaxPool1d(max_pool_size)

        self.RNNModel = RNNModel(num_filters,hidden_dim,rnn_bidirectional,num_layers=rnn_num_layers,rnn_dropout=rnn_dropout,fc_dropout=dropout)


    def forward(self,x,x_len):
        
        x = x.permute(0,2,1)
        conv_x = F.relu(self.conv(x))
        pooled_x = self.pool(conv_x)

        new_x = pooled_x.permute(0,2,1)
        #print(new_x.shape)
        new_x_len = length_after_1d_convolution(x_len,self.filter_size,self.max_pool_size)
        #print(new_x_len)

        ans = self.RNNModel(new_x,new_x_len)
        return ans
ensemble_model = EnsembleModel()


In [12]:
from torch.utils.tensorboard import SummaryWriter
from datetime import  datetime

def train(model,train_dataloader,valid_dataloader,num_epochs,criterion,optimizer,check_point_name,tensorboard_name,device_train = device_fast,use_rnn = False):
    
    current_datetime = datetime.now().strftime("%d/%m/%Y_%H:%M:%S")
    tensorboard_name = tensorboard_name + "_" + current_datetime
    writer = SummaryWriter('runs/' + tensorboard_name)
    model = model.to(device_train)


    clip = 0
    if use_rnn:
        clip = 5

    best_validation_loss = 1000.0
    valdiation_loss_not_decreased_steps = 0
    
    model.train()
    for e in range(num_epochs):
        
        training_set_size = 0
        training_loss = 0.0
        model.train()

        for data in tqdm(train_dataloader):
            
            optimizer.zero_grad()
            input_reviews,inp_lengths,output_labels = data['input'], data['lengths'],data['labels']
            input_reviews = input_reviews.to(device_train)
            training_set_size += input_reviews.shape[0]
            output = model(input_reviews,inp_lengths)
            output = output.to(device_cpu)
            loss = criterion(output,output_labels.float())
            training_loss += loss.item()
            loss.backward()
            if use_rnn:
                nn.utils.clip_grad_norm_(model.parameters(),clip)
            optimizer.step()
        
        current_training_loss = training_loss / training_set_size
        print("Epoch " + str(e) + " Average Training Loss = " +  str(current_training_loss))
        writer.add_scalars(tensorboard_name + '; Loss vs Epoch',{'train' : current_training_loss},e)
        
        model.eval()
        
        if valid_dataloader is None:
            continue
        
        validation_set_size  = 0 
        if e% VALIDATION_LOSS_COMPUTE_STEP==0:
            correct_count = 0
            validation_loss = 0

            for i,data in enumerate(valid_dataloader,0):
                input_reviews,inp_lengths,output_labels = data['input'], data['lengths'],data['labels']
                input_reviews = input_reviews.to(device_train)
                validation_set_size += input_reviews.shape[0]
                output = model(input_reviews,inp_lengths)
                output = output.to(device_cpu)
                loss = criterion(output,output_labels.float())
                validation_loss += loss.item()
                nearest_class = torch.round(output)

                correct = (nearest_class == output_labels.float()).float()
                correct_count += correct.sum()
            correct_count = int(correct_count)
            current_validation_loss = (1.0* validation_loss)/validation_set_size
            print("Epoch " + str(e) + " " +  "Validation Loss = " + str(current_validation_loss) )
            print("Validation Set Accuracy = " + str((correct_count/validation_set_size)*100) )

            writer.add_scalar(tensorboard_name + ' Validation Accuracy vs Epoch ',int((correct_count/validation_set_size)*100),e)
            writer.add_scalars('Loss vs Epoch',{'valid' : current_validation_loss},e)
            if current_validation_loss < best_validation_loss:
                valdiation_loss_not_decreased_steps = 0
                torch.save(model.state_dict(),check_point_name)
                best_validation_loss = current_validation_loss
            else:
                valdiation_loss_not_decreased_steps +=1
        
        if valdiation_loss_not_decreased_steps >= PATIENCE_PARAMETER:
            break

In [11]:
EPOCHS = 50
dan = DAN(train_device=device_cpu)
criterion = nn.BCELoss()
LR = 0.01
optimizer = optim.Adam(dan.parameters(),lr=LR)

model_name = type(dan).__name__
optimizer_name = type(optimizer).__name__
loss_fn_name = type(criterion).__name__

checkpoint_name = 'checkpoints/'+model_name+ '_' + str(EPOCHS)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(LR)+'_.pth'  
train(dan,train_dataloader,valid_dataloader,EPOCHS,criterion,optimizer,checkpoint_name,model_name,device_cpu,use_rnn=False)

100%|██████████| 500/500 [00:41<00:00, 12.19it/s]


Epoch 0 Average Training Loss = 0.006330133778043092
Epoch 0 Validation Loss = 0.005358372364193201
Validation Set Accuracy = 85.5875


100%|██████████| 500/500 [00:45<00:00, 10.92it/s]


Epoch 1 Average Training Loss = 0.005534328098408878
Epoch 1 Validation Loss = 0.005460455451160669
Validation Set Accuracy = 84.6625


100%|██████████| 500/500 [00:45<00:00, 11.06it/s]


...

Epoch 19 Average Training Loss = 0.004709118981845677
Epoch 19 Validation Loss = 0.005087011504918337
Validation Set Accuracy = 86.47500000000001


100%|██████████| 500/500 [00:40<00:00, 12.34it/s]


Epoch 20 Average Training Loss = 0.0047282912386581305
Epoch 20 Validation Loss = 0.005033512288704515
Validation Set Accuracy = 86.2125


100%|██████████| 500/500 [00:44<00:00, 11.34it/s]


Epoch 21 Average Training Loss = 0.004699433033820242
Epoch 21 Validation Loss = 0.005492307219654322
Validation Set Accuracy = 85.1625


In [20]:
EPOCHS = 50
BIDIRECTIONAL = 'F'
NUM_LAYERS = 1
bidirectional = True if BIDIRECTIONAL=='T' else False
RNN_TYPE = 'gru'
RNN_DROPOUT = 0.3 if NUM_LAYERS == 1 else 0.0

model = RNNModel(rnn_type=RNN_TYPE,bidirectional=bidirectional,num_layers=NUM_LAYERS,rnn_dropout=RNN_DROPOUT)
criterion = nn.BCELoss()
LR = 0.001
optimizer = optim.Adam(dan.parameters(),lr=LR)

model_name = type(model).__name__
optimizer_name = type(optimizer).__name__
loss_fn_name = type(criterion).__name__

checkpoint_name = 'checkpoints/'+model_name+ '_' +RNN_TYPE + '_'+ str(EPOCHS)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(LR) + "_" + str(NUM_LAYERS) +'_'+ BIDIRECTIONAL +'_' + str(RNN_DROPOUT)+'_.pth' 
train(model,train_dataloader,valid_dataloader,EPOCHS,criterion,optimizer,checkpoint_name,model_name,device_cpu,use_rnn=False)

  0%|          | 1/500 [00:09<1:19:23,  9.55s/it]


KeyboardInterrupt: 

In [22]:
EPOCHS = 50
model = CNNModel()
criterion = nn.BCELoss()
LR = 0.01
optimizer = optim.Adam(dan.parameters(),lr=LR)

model_name = type(model).__name__
optimizer_name = type(optimizer).__name__
loss_fn_name = type(criterion).__name__

checkpoint_name = 'checkpoints/'+model_name+ '_' + str(EPOCHS)+"_"+ loss_fn_name  +"_"+ optimizer_name + "_" + str(LR) +'_.pth' 
train(model,train_dataloader,valid_dataloader,EPOCHS,criterion,optimizer,checkpoint_name,model_name,device_cpu,use_rnn=False)

  0%|          | 2/500 [00:19<1:20:13,  9.67s/it]


KeyboardInterrupt: 

In [26]:
EPOCHS = 50
BIDIRECTIONAL = 'T'
NUM_LAYERS = 1
bidirectional = True if BIDIRECTIONAL=='T' else False
RNN_DROPOUT = 0.3 if NUM_LAYERS == 1 else 0.0


model = EnsembleModel(rnn_bidirectional=bidirectional,rnn_num_layers=NUM_LAYERS,rnn_dropout=RNN_DROPOUT)
criterion = nn.BCELoss()
LR = 0.001
optimizer = optim.Adam(dan.parameters(),lr=LR)

model_name = type(model).__name__
optimizer_name = type(optimizer).__name__
loss_fn_name = type(criterion).__name__

checkpoint_name = 'checkpoints/' + model_name +'_' + str(EPOCHS)+'_'+ loss_fn_name  +'_'+ optimizer_name + '_' + str(LR) + '_' + str(NUM_LAYERS)+ '_' + BIDIRECTIONAL +'_' + str(RNN_DROPOUT) +'_.pth' 
train(model,train_dataloader,valid_dataloader,EPOCHS,criterion,optimizer,checkpoint_name,model_name,device_cpu,use_rnn=False)

  0%|          | 2/500 [00:29<2:01:55, 14.69s/it]


KeyboardInterrupt: 

### HyperParameter Search

In [ ]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [ ]:
def test(checkpoint_name,test_data,test_lengths,test_labels):
    model_name = checkpoint_name[14:]
    parameters =  model_name.split('_')
    count = 0
    model = None
    if parameters[0]=='DAN':
        model = DAN()
    elif parameters[0] == 'CNNModel':
        model = CNNModel()
    elif parameters[0] == 'RNNModel':
        bidir = True if parameters[7]=='T' else False
        model = RNNModel(num_layers=int(parameters[6]),rnn_type=parameters[1],bidirectional=bidir)
    elif parameters[0] == 'EnsembleModel':
        bidir = True if parameters[7] =='T' else False
        num_layers = int(parameters[6])
        model = EnsembleModel(rnn_bidirectional=bidir,rnn_num_layers=num_layers,rnn_dropout=float(parameters[8]))
    
    model.load_state_dict(torch.load(checkpoint_name))
    model.eval()
    for i in range(len(test_data)):
        ans = model(test_data[i],[test_lengths[i]])
        ans = torch.round(ans)
        if ans[0][0] == test_labels[i]:
            count+=1
    
    print("Accuracy = " + str(count/len(test_data)))


In [ ]:
test_dataset_labels = []  
test_processed_text = []
with open("./Train dataset.csv") as csvfile:
    csvFile = csv.reader(csvfile)
    next(csvFile)
    for line in csvFile:
        processed_text = preprocess_text(line[0])
        label = 1.0 if line[1] == 'positive' else 0.0
        test_dataset_labels.append(label)
        test_processed_text.append(processed_text)



test_word_embeddings = []
test_sentence_lengths = []
for text in test_processed_text:
    embeddings,length = getWordEmbeddingforText(text)
    test_sentence_lengths.append(length)
    test_word_embeddings.append(embeddings.unsqueeze(0))



In [ ]:
test('./checkpoints/dan_50_bce_adam_0.01',test_word_embeddings,test_sentence_lengths,test_dataset_labels)    

In [16]:
embeddings,length = getWordEmbeddingforText("the movie is not that good.")
dan = DAN()
dan.load_state_dict(torch.load('./checkpoints/DAN_50_BCELoss_Adam_0.01_.pth'))
dan.eval()
dan(embeddings.unsqueeze(0),[length])

tensor([[0.2348]], grad_fn=<SigmoidBackward0>)